In [99]:
using Pkg
Pkg.activate("..")
Pkg.add("BenchmarkTools")
using Test
using BenchmarkTools

BenchmarkTools.DEFAULT_PARAMETERS.samples = 100000
BenchmarkTools.DEFAULT_PARAMETERS.seconds = 10;

  Activating project at `c:\Users\wenbl13\Desktop\Coding-Club\coding club`
   Resolving package versions...
  No Changes to `C:\Users\wenbl13\Desktop\Coding-Club\coding club\Project.toml`
  No Changes to `C:\Users\wenbl13\Desktop\Coding-Club\coding club\Manifest.toml`


# Approach 1

In [47]:
function pow_ver1(x::Float64, n::Int32)
    if n>=0
        return pow_ver1_rec(x, n)
    end
    return 1.0 / pow_ver1_rec(x, -n)
end

function pow_ver1_rec(x::Float64, n::Int32)
    # base case
    if n==0
        return 1.0
    end
    if n==1
        return x
    end
    # rec
    n_half_rslt = pow_ver1_rec(x, floor(Int, n/2))
    residue = n%2 == 1 ? x : 1
    return residue * n_half_rslt * n_half_rslt;
end

pow_ver1_rec (generic function with 3 methods)

Test correctness

In [53]:
x = 1.0
n = Int32(-2147483648)
@test pow_ver1(x, n) == x^n

for i = 1:10000
    x = rand()*10000
    n = Int32(rand(-2147483648 : 2147483647))
    @test pow_ver1(x, n) == x^n
end

Error During Test at c:\Users\wenbl13\Desktop\Coding-Club\coding club\2\pow.ipynb:3
  Test threw exception
  Expression: pow_ver1(x, n) == x ^ n
  StackOverflowError:
  Stacktrace:
    [1] pow_ver1_rec(x::Float64, n::Int64)
      @ Main c:\Users\wenbl13\Desktop\Coding-Club\coding club\2\pow.ipynb:8
    [2] pow_ver1_rec(x::Float64, n::Int64) (repeats 37296 times)
      @ Main c:\Users\wenbl13\Desktop\Coding-Club\coding club\2\pow.ipynb:17
    [3] pow_ver1(x::Float64, n::Int32)
      @ Main c:\Users\wenbl13\Desktop\Coding-Club\coding club\2\pow.ipynb:5
    [4] eval
      @ .\boot.jl:368 [inlined]
    [5] include_string(mapexpr::typeof(REPL.softscope), mod::Module, code::String, filename::String)
      @ Base .\loading.jl:1428
    [6] #invokelatest#2
      @ .\essentials.jl:729 [inlined]
    [7] invokelatest
      @ .\essentials.jl:726 [inlined]
    [8] (::VSCodeServer.var"#198#199"{VSCodeServer.NotebookRunCellArguments, String})()
      @ VSCodeServer c:\Users\wenbl13\.vscode\extensions\

Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")

# Approach 2

In [55]:
function pow_ver2(x::Float64, n::Int32)
    rslt = pow_ver2_rec(x, n)
    if n<0
        rslt = 1.0 / rslt
    end
    return rslt
end

function pow_ver2_rec(x::Float64, n::Int32)
    # base case
    if n==0
        return 1.0
    end
    if n==1 || n ==-1
        return x
    end
    # rec
    n_half_rslt = pow_ver2_rec(x, floor(Int, n/2))
    residue = n%2 == 1 ? x : 1
    return residue * n_half_rslt * n_half_rslt;
end

pow_ver2_rec (generic function with 3 methods)

Test correctness

In [56]:
x = 1.0
n = Int32(-2147483648)
@test pow_ver2(x, n) == x^n

for i = 1:10000
    x = rand()*10000
    n = Int32(rand(-2147483648 : 2147483647))
    @test pow_ver2(x, n) == x^n
end

# Approach 3

In [72]:
function pow_ver3(x::Float64, n::Int32)
    # edge cases
    if x == 1.0 || n == 0
        return 1.0
    end

    # special case
    if n == -2147483648
        i = 31
        while i >= 0
            x *= x
            # out of double range
            if x == Inf
                return 0.0
            end
            i -= 1
        end
        return x
    end

    # general cases
    is_neg = false
    if n < 0
        is_neg = true
        n = ~n + 1
    end

    rslt = 1.0
    curr_pow = x

    while n > 0
        if n & 1 == 1
            rslt *= curr_pow
        end
        curr_pow *= curr_pow
        n >>= 1
    end

    return is_neg ? 1.0 / rslt : rslt
end


pow_ver3 (generic function with 2 methods)

In [95]:
function _special_case(x::Float64)
    i = 32
    while (i-=1) >= 0
        x *= x
        # out of double range
        x == Inf && (return 0.0)
    end
    return x
end
function pow_optimized(x::Float64, n::Int32)
    # edge cases
    (x == 1.0 || n == 0) && (return 1.0)

    # special case
    n == -2147483648 && (return _special_case(x))

    # general cases
    is_neg = n < 0
    n = is_neg ? ~n + 1 : n 
    rslt = 1.0
    while n > 0
        n & 1 == 1 && (rslt *= x)
        x *= x
        n >>= 1
    end

    return is_neg ? 1.0 / rslt : rslt
end


pow_optimized (generic function with 1 method)

In [79]:
x = 1.0
n = Int32(-2147483648)
@test pow_ver3(x, n) == x^n

for i = 1:10000
    x = rand()*10000
    n = Int32(rand(-2147483648 : 2147483647))
    @test pow_ver3(x, n) == x^n
end

# Benchmark

In [100]:
x = rand()*10000
n = Int32(rand(-2147483648 : 2147483647));

In [101]:
@benchmark pow_ver1(x, n)

BenchmarkTools.Trial: 86006 samples with 460 evaluations.
 Range (min … max):  227.174 ns …  1.037 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     242.826 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   249.431 ns ± 26.883 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▄▇▆█▆▅▃▂▁▁  ▁▂▂▁▁▁▁▁        ▁▁▁                           ▃
  ▇▅▆███████████████████████████████████████████████▇▇▇▇▇▇▇▇▇▇ █
  227 ns        Histogram: log(frequency) by time       375 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [102]:
@benchmark pow_ver2(x, n)

BenchmarkTools.Trial: 88819 samples with 438 evaluations.
 Range (min … max):  230.137 ns …  1.854 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     246.804 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   253.300 ns ± 24.325 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▃▃▇█▆▅▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁                                  ▃
  ▆▇▆▆███████████████████████████████████████▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▆ █
  230 ns        Histogram: log(frequency) by time       365 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [103]:
@benchmark pow_ver3(x, n)

BenchmarkTools.Trial: 100000 samples with 996 evaluations.
 Range (min … max):  26.004 ns … 373.795 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     27.610 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   28.501 ns ±   4.373 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▁▄██▇▅▃▂▁                            ▁▁       ▁▁            ▃
  ▇██████████▇▆▆▆▅▅▅▅▆▆▇▇▆▇▆▆▇██▇▇▇▆▆▅▇█████▇▆▇▇████▇▇▇▆▅▅▅▅▅▅ █
  26 ns         Histogram: log(frequency) by time      48.9 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [104]:
@benchmark pow_optimized(x, n)

BenchmarkTools.Trial: 100000 samples with 996 evaluations.
 Range (min … max):  24.699 ns … 306.225 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     26.205 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   27.170 ns ±   4.203 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁▄▃███▄▄▄▂                             ▁▁         ▁          ▃
  ████████████▇▆▅▅▆▆▆▆▆▆▆▆▇▇▆▇▇██▇▇▇▇▇▆▆██████▇▇▆██████▇▆▆▆▅▆▄ █
  24.7 ns       Histogram: log(frequency) by time      46.5 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [105]:
@benchmark x^n

BenchmarkTools.Trial: 100000 samples with 984 evaluations.
 Range (min … max):  59.045 ns …  1.260 μs  ┊ GC (min … max): 0.00% … 90.87%
 Time  (median):     61.789 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   64.497 ns ± 16.506 ns  ┊ GC (mean ± σ):  0.38% ±  1.57%

  ▂▆▅█▅▆▃▄▄▅▃▃▁▁▁▁▁▁   ▁▁▁▁▁▁▁▁▁                              ▃
  █████████████████████████████████▇█▇▇█▇██▇█▇██████▇▇▇▇▇▆▆▇▆ █
  59 ns        Histogram: log(frequency) by time       103 ns <

 Memory estimate: 16 bytes, allocs estimate: 1.